In [1]:
#________Packages_____________________________________________________________________________
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from ttkbootstrap.dialogs import Messagebox
from tkinter import filedialog
import tkinter as tk
import pandas as pd
import csv
import re
import psycopg2
from datetime import datetime
import emoji

#CONNECTION TO DATABASE
def data_base_connection():
    return psycopg2.connect(host="dbod-ot-hybrids-test-system.cern.ch",port=6603,user="admin",password="1ireang4",dbname="ot_hybrids_testsystem",connect_timeout=10)



#________Format and other different Checks__________________________________________________________________________

superbarcode_format = re.compile(
    r"^(?:"
        # FEH/POH/SEH — unchanged rule: -2PP + 6 alphanumerics
        r"(?:PSFEH(?:16|26|40)(?:R|L)|2SFEH(?:18|40)(?:R|L)|(?:PS|2S)POH|(?:PS|2S)SEH)"
        r"-2(?:1[4-9]|[2-9]\d)[A-Z0-9]{6}"
    r"|"
        # ROH — TT in {16,26,40}, PP in 14–99, LL in {05,10}, then 4 alphanumerics
        r"(?:PS|2S)ROH(?:16|26|40)"
        r"-2(?:1[4-9]|[2-9]\d)(?:05|10)[A-Z0-9]{4}"
    r")$"
)


serial_number_format = re.compile(
    r"^(?:"
        # FEH/POH/SEH — keep simple: hyphen + 9 digits
        r"(?:PSFEH(?:16|26|40)(?:R|L)|2SFEH(?:18|40)(?:R|L)|(?:PS|2S)POH|(?:PS|2S)SEH)"
        r"-\d{9}"
    r"|"
        # ROH — TT in {16,26,40}, LL in {05,10}; numeric tail must mirror TT
        r"(?:PS|2S)ROH(16|26|40)"
        r"-2\1(?:05|10)\d{4}"
    r")$"
)


delivery_number_format = re.compile(r"^(PS\d{3}|2S\d{3})$")
valtronic_lot_code_format = re.compile(r"^\d{10}$")


#________Storage______________________________________________________________________________
reception_info = {}
superbarcodes = []
hybrid_barcodes = []



#________Warning STUFF______________________________________________________________________
def warning_popup_multiple(invalid_codes, storage_list, display_listbox, added_this_scan=None):
    if not invalid_codes:
        return
    win = ttk.Toplevel()
    win.title("Format Warning - Multiple Codes")
    win.geometry("400x300")
    msg = "The following codes have the WRONG FORMAT:\n\n" + "\n".join(invalid_codes)
    ttk.Label(win, text=msg, foreground="red", wraplength=380, justify="left").pack(pady=10)

    def keep_all():
        if added_this_scan:
            # do nothing to added valid ones; they stay
            pass
        win.destroy()

    def discard_all():
        # remove *only what we added in this scan*
        if added_this_scan:
            # remove from storage list
            for code, _ in added_this_scan:
                try:
                    storage_list.remove(code)
                except ValueError:
                    pass
            # remove from listbox (delete by index descending to avoid shifting)
            for _, idx in sorted(added_this_scan, key=lambda t: t[1], reverse=True):
                try:
                    display_listbox.delete(idx)
                except Exception:
                    pass
        win.destroy()

    ttk.Button(win, text="Keep All Anyway", command=keep_all, bootstyle="warning").pack(side="left", padx=20, pady=10)
    ttk.Button(win, text="Discard All", command=discard_all, bootstyle="secondary").pack(side="right", padx=20, pady=10)


def duplicate_warning_popup_multiple(duplicates, storage_list, display_listbox):
    if not duplicates:
        return
    win = ttk.Toplevel()
    win.title("Duplicate Warning - Multiple Codes")
    win.geometry("400x300")
    msg = "The following codes are already scanned:\n\n" + "\n".join(duplicates)
    ttk.Label(win, text=msg, foreground="orange", wraplength=380, justify="left").pack(pady=10)

    def keep_all():
        for code in duplicates:
            storage_list.append(code)
            display_listbox.insert(tk.END, code)
        win.destroy()

    def discard_all():
        win.destroy()

    ttk.Button(win, text="Keep All Duplicates", command=keep_all, bootstyle="warning").pack(side="left", padx=20, pady=10)
    ttk.Button(win, text="Discard Duplicates", command=discard_all, bootstyle="secondary").pack(side="right", padx=20, pady=10)

#------------------------------
#FIRST BUTTOM
#------------------------------

#________Reception B02 Popup__________________________________________________________________
def reception_b02_popup():
    popup = ttk.Toplevel()
    popup.title("Reception in B02 - Required Information")
    popup.geometry("400x420")

    ttk.Label(popup, text="Reception Number (PSXXX or 2SXXX)").pack(pady=5)
    reception_entry = ttk.Entry(popup)
    reception_entry.pack(pady=5)

    ttk.Label(popup, text="Reception Date (DD.MM.YYYY)").pack(pady=5)
    date_entry = ttk.Entry(popup)
    date_entry.pack(pady=5)

    def format_date_entry(event):
        raw = date_entry.get().replace(".", "")
        if len(raw) > 2:
            raw = raw[:2] + "." + raw[2:]
        if len(raw) > 5:
            raw = raw[:5] + "." + raw[5:]
        date_entry.delete(0, tk.END)
        date_entry.insert(0, raw)

    date_entry.bind("<KeyRelease>", format_date_entry)

    ttk.Label(popup, text="Total Number of Hybrid Pieces").pack(pady=5)
    pcs_entry = ttk.Entry(popup)
    pcs_entry.pack(pady=5)

    ttk.Label(popup, text="Valtronic Lot Code (Optional)").pack(pady=5)
    lot_entry = ttk.Entry(popup)
    lot_entry.pack(pady=5)

    def proceed():
        reception = reception_entry.get().strip().upper()
        date = date_entry.get().strip()
        pcs = pcs_entry.get().strip()
        lot = lot_entry.get().strip()

        if not delivery_number_format.match(reception):
            Messagebox.show_error("Reception number must be in the format PSXXX or 2SXXX. TRY AGAIN")
            return
        try:
            datetime.strptime(date, "%d.%m.%Y")
        except ValueError:
            Messagebox.show_error("Date must be in format DD.MM.YYYY. TRY AGAIN")
            return
        if not pcs.isdigit():
            Messagebox.show_error("Total number of pieces must be a numeric value. TRY AGAIN")
            return
        if lot and not valtronic_lot_code_format.match(lot):
            Messagebox.show_error("Valtronic lot code must be a 10-digit number. TRY AGAIN")
            return

        global reception_info
        reception_info = {
            "reception_number": reception,
            "reception_date": date,
            "total_pcs": int(pcs),
            "valtronic_lot_code": lot if lot else None
        }

        popup.destroy()
        scan_superbarcode_popup()

    ttk.Button(popup, text="Next Step", command=proceed, bootstyle="success").pack(pady=15)

#¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤
Delay = 350   # THE CODE SHOULD ONLY DO SOMETHING WITH THE SCANNED STUFF AFTER THE AMOUNT OF MS
#¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤¤


#________Superbarcode Scanner Popup___________________________________________________________
def scan_superbarcode_popup():
    popup = ttk.Toplevel()
    popup.title("Scan Superbarcodes")
    popup.geometry("700x500")

    display_listbox = tk.Listbox(popup, height=8, width=60, font=("Segoe UI", 12))
    display_listbox.pack(pady=10)

    ttk.Label(popup, text="Scan Superbarcodes here").pack()

    # ---- use Entry instead of Text + add debounce ----
    scan_var = tk.StringVar()
    scan_entry = ttk.Entry(popup, textvariable=scan_var, width=50)
    scan_entry.pack()
    scan_entry.focus_set()
    scan_entry.after(10, scan_entry.focus_force)

    debounce_job = {"id": None}  # mutable holder so inner functions can modify

    def process_scan(text):
        # --------- moved your original body here (minus the .get()/.delete()) ----------
        raw_input = text.strip()
        if not raw_input:
            return

        raw_input = raw_input.upper()
        raw_input = re.sub(r"TRAY\s*\d+", "", raw_input, flags=re.IGNORECASE)
        raw_input = re.sub(r"(PS|2S)FEH(16|26|40)(R|L)", r"\1FEH\2\3", raw_input)

        matches = re.finditer(
            r"((?:PSFEH(?:16|26|40)(?:R|L)|2SFEH(?:18|40)(?:R|L)|(?:PS|2S)POH|(?:PS|2S)ROH(?:16|26|40)|(?:PS|2S)SEH)[\s\+\-]?(?:\d{9}|2PP\w{5,}))",
            raw_input
        )

        valid_codes = []
        duplicate_codes = []
        all_matched_originals = []
        added_this_scan = []

        for match in matches:
            raw_match = match.group(1)
            normalized_code = re.sub(r"[\+\s]", "-", raw_match)
            all_matched_originals.append(raw_match)

            if superbarcode_format.match(normalized_code):
                if normalized_code in superbarcodes:
                    duplicate_codes.append(normalized_code)
                else:
                    superbarcodes.append(normalized_code)
                    idx = display_listbox.size()
                    display_listbox.insert(tk.END, normalized_code)
                    added_this_scan.append((normalized_code, idx))
                valid_codes.append(normalized_code)
            else:
                print(f"{emoji.emojize(':worried_face:')} Invalid format: {normalized_code}")

        # find invalid fragments
        raw_cleaned = raw_input
        for text in all_matched_originals:
            raw_cleaned = raw_cleaned.replace(text, "")
        invalid_codes = [frag for frag in re.split(r"[+\s]+", raw_cleaned) if frag.strip()]

        warning_popup_multiple(invalid_codes, superbarcodes, display_listbox, added_this_scan)
        duplicate_warning_popup_multiple(duplicate_codes, superbarcodes, display_listbox)

    def do_process_now():
        # cancel any pending debounce, then process current content once
        if debounce_job["id"] is not None:
            popup.after_cancel(debounce_job["id"])
            debounce_job["id"] = None
        text = scan_var.get()
        if text.strip():
            process_scan(text)
            scan_var.set("")  # clear only AFTER processing finishes

    def on_key_release(_event=None):
        # reschedule debounce on every keystroke
        if debounce_job["id"] is not None:
            popup.after_cancel(debounce_job["id"])
        # 200 ms is a good starting point; tweak if needed
        debounce_job["id"] = popup.after(Delay, do_process_now)

    def on_return(event=None):
        # scanners usually send Enter at the end — process immediately
        do_process_now()
        return "break"  # prevent default beep

    scan_entry.bind("<KeyRelease>", on_key_release)
    scan_entry.bind("<Return>", on_return)

    def finish():
        popup.destroy()
        scan_hybrids_popup()

    # Button remains, but it waits a beat before reading (in case user clicks too soon)
    ttk.Button(
        popup,
        text="Scan next Superbarcode",
        bootstyle="primary",
        command=lambda: popup.after(Delay, do_process_now)
    ).pack(pady=5)

    ttk.Button(
        popup,
        text="Finish and move to Individual Hybrid Scanning",
        command=finish,
        bootstyle="success"
    ).pack(pady=5)


#________Individual Hybrid Scanner Popup______________________________________________________
def scan_hybrids_popup():
    popup = ttk.Toplevel()
    popup.title("Scan Individual Hybrids")
    popup.geometry("700x500")

    display_listbox = tk.Listbox(popup, height=8, width=60, font=("Segoe UI", 12))
    display_listbox.pack(pady=10)

    ttk.Label(popup, text="Scan Hybrid Serial here").pack()

    scan_var = tk.StringVar()
    scan_entry = ttk.Entry(popup, textvariable=scan_var, width=50)
    scan_entry.pack()
    scan_entry.focus_set()
    scan_entry.after(10, scan_entry.focus_force)

    debounce_job = {"id": None}

    def process_scan(text):
        raw_input = text.strip().upper()
        if not raw_input:
            return

        raw_input = re.sub(r"TRAY\s*\d+", "", raw_input, flags=re.IGNORECASE)

        matches = re.finditer(
            r"((?:PSFEH(?:16|26|40)(?:R|L)|2SFEH(?:18|40)(?:R|L)|(?:PS|2S)POH|(?:PS|2S)ROH(?:16|26|40)|(?:PS|2S)SEH)[\s\+\-]?\d{9})",
            raw_input
        )

        valid_codes = []
        duplicate_codes = []
        all_matched_full_codes = []
        added_this_scan = []

        for match in matches:
            full_code = match.group(1)
            normalized_code = re.sub(r"[\+\s]", "-", full_code)
            all_matched_full_codes.append(match.group(0))

            if serial_number_format.match(normalized_code):
                if normalized_code in hybrid_barcodes:
                    duplicate_codes.append(normalized_code)
                else:
                    hybrid_barcodes.append(normalized_code)
                    idx = display_listbox.size()
                    display_listbox.insert(tk.END, normalized_code)
                    added_this_scan.append((normalized_code, idx))
                valid_codes.append(normalized_code)

        # invalid fragments
        raw_cleaned = raw_input
        for code in all_matched_full_codes:
            raw_cleaned = raw_cleaned.replace(code.replace("-", ""), "")
        invalid_codes = [frag for frag in re.split(r"[+\s]+", raw_cleaned) if frag.strip()]

        warning_popup_multiple(invalid_codes, hybrid_barcodes, display_listbox, added_this_scan)
        duplicate_warning_popup_multiple(duplicate_codes, hybrid_barcodes, display_listbox)

    def do_process_now():
        if debounce_job["id"] is not None:
            popup.after_cancel(debounce_job["id"])
            debounce_job["id"] = None
        text = scan_var.get()
        if text.strip():
            process_scan(text)
            scan_var.set("")

    def on_key_release(_event=None):
        if debounce_job["id"] is not None:
            popup.after_cancel(debounce_job["id"])
        debounce_job["id"] = popup.after(Delay, do_process_now)

    def on_return(event=None):
        do_process_now()
        return "break"

    scan_entry.bind("<KeyRelease>", on_key_release)
    scan_entry.bind("<Return>", on_return)

    def finish():
        popup.destroy()
        show_report()

    ttk.Button(
        popup,
        text="Scan next Hybrid Serial",
        bootstyle="primary",
        command=lambda: popup.after(Delay, do_process_now)
    ).pack(pady=5)

    ttk.Button(popup, text="Finish and show Report", command=finish, bootstyle="success").pack(pady=5)


#________Report Page__________________________________________________________________________
def show_report():
    report_window = ttk.Toplevel()
    report_window.title("Reception Report")
    report_window.geometry("800x700")

    # ----list dialog with buttons; returns a key like "add" / "skip" ----
    def show_list_dialog(title, header, items, buttons):
        win = ttk.Toplevel(report_window)
        win.title(title)
        win.geometry("600x500")
        win.transient(report_window)
        win.grab_set()

        ttk.Label(win, text=header, wraplength=560, justify="left").pack(padx=10, pady=10, anchor="w")

        frame = ttk.Frame(win); frame.pack(fill="both", expand=True, padx=10)
        lb = tk.Listbox(frame, height=12)
        lb.pack(side="left", fill="both", expand=True)
        sb = ttk.Scrollbar(frame, orient="vertical", command=lb.yview); sb.pack(side="right", fill="y")
        lb.configure(yscrollcommand=sb.set)

        for it in items:
            lb.insert(tk.END, str(it))

        btn_row = ttk.Frame(win); btn_row.pack(pady=10)
        result = {"value": None}
        for text, key, style in buttons:
            ttk.Button(btn_row, text=text, bootstyle=style,
                       command=lambda k=key: (result.update(value=k), win.destroy())).pack(side="left", padx=8)

        win.wait_window()
        return result["value"]

    # ---- database checks ----
    disallowed_serials = []
    missing_from_db = []

    try:
        conn = data_base_connection()
        cursor = conn.cursor()

        # Hybrids marked as NOT allowed to ship
        cursor.execute("""
            SELECT serial_number
            FROM shipping_tool.shipping_data
            WHERE serial_number = ANY(%s) AND allowed_to_ship = FALSE
        """, (hybrid_barcodes,))
        disallowed_serials = [row[0] for row in cursor.fetchall()]

        # Which of the scanned serials exist at all?
        cursor.execute("""
            SELECT serial_number
            FROM shipping_tool.shipping_data
            WHERE serial_number = ANY(%s)
        """, (hybrid_barcodes,))
        found_serials = set(row[0] for row in cursor.fetchall())
        missing_from_db = [s for s in hybrid_barcodes if s not in found_serials]

        conn.close()
    except Exception as e:
        disallowed_serials.append(f"[DB error: {str(e)}]")

    # ---- If some scanned serials are NOT in the database, ask to add them ----
    if missing_from_db:
        choice = show_list_dialog(
            "Add missing serials?",
            f"{len(missing_from_db)} scanned serial(s) are not in the database.\n"
            f"Do you want to add them now (other columns will be left empty)?",
            missing_from_db,
            buttons=[("Add all to database", "add", "success"),
                     ("Skip (do not add)", "skip", "secondary")]
        )
        if choice == "add":
            try:
                conn = data_base_connection()
                cursor = conn.cursor()
                # Inserts only serial_number; other fields remain NULL/default
                cursor.executemany("""
                    INSERT INTO shipping_tool.shipping_data (serial_number)
                    VALUES (%s)
                    ON CONFLICT (serial_number) DO NOTHING
                """, [(s,) for s in missing_from_db])
                conn.commit()
                conn.close()
                Messagebox.show_info(f"Added {len(missing_from_db)} serial(s) to the database.")
                
                missing_from_db = []
            except Exception as e:
                Messagebox.show_error(f"Failed to add missing serials:\n{str(e)}")

    # ---- Build of report text ----
    expected = reception_info.get("total_pcs", 0)
    actual = len(hybrid_barcodes)
    super_count = len(superbarcodes)
    delivery = reception_info.get("reception_number", "")

    mismatched_hybrids = [h for h in hybrid_barcodes if h not in superbarcodes]
    mismatched_superbarcodes = [s for s in superbarcodes if s not in hybrid_barcodes]

    summary_text = f"""
E-log template:
The following {expected} Number of Hybrids from Delivery {delivery} were taken for QC in B04.
Total scanned Hybrid Serials: {actual}
Total scanned Superbarcodes: {super_count}
"""

    if not mismatched_hybrids and not mismatched_superbarcodes and not disallowed_serials and not missing_from_db:
        summary_text += f"\n{emoji.emojize(':check_mark:')} {emoji.emojize(':star-struck:')} All checks OK.\n"
    else:
        summary_text += f"\n{emoji.emojize(':warning:')} Results:\n"

        if disallowed_serials:
            summary_text += (
                f"\n{emoji.emojize(':double_exclamation_mark:')} "
                f"{len(disallowed_serials)} hybrid(s) are marked as 'NOT ALLOWED TO SHIP' in the database:\n"
                + "\n".join(disallowed_serials) + "\n"
            )

        if missing_from_db:
            summary_text += (
                f"\n{emoji.emojize(':downcast_face_with_sweat:')} "
                f"{len(missing_from_db)} hybrid(s) were NOT FOUND in the database:\n"
                + "\n".join(missing_from_db) + "\n"
            )

        if mismatched_hybrids:
            summary_text += f"\nHybrid Serials not found among the Superbarcodes ({len(mismatched_hybrids)}):\n"
            summary_text += "\n".join(mismatched_hybrids) + "\n"

        if mismatched_superbarcodes:
            summary_text += f"\nSuperbarcodes not found among Hybrid Serials ({len(mismatched_superbarcodes)}):\n"
            summary_text += "\n".join(mismatched_superbarcodes) + "\n"

    # ----report text + copy/close ----
    text_box = tk.Text(report_window, wrap="word", height=20, font=("Segoe UI", 11))
    text_box.insert(tk.END, summary_text.strip())
    text_box.pack(padx=10, pady=10, fill="both", expand=True)

    def copy_to_clipboard():
        report_window.clipboard_clear()
        report_window.clipboard_append(summary_text.strip())
        report_window.update()

    btn_frame = ttk.Frame(report_window); btn_frame.pack(pady=10)
    ttk.Button(btn_frame, text="Copy all Text to Clipboard", command=copy_to_clipboard, bootstyle="info").pack(side="left", padx=10)
    ttk.Button(btn_frame, text="Close", command=report_window.destroy, bootstyle="danger").pack(side="right", padx=10)



#------------------------------
#SECOND BUTTOM
#------------------------------

#________IMPORT AND MODIFY______________________________________________________
def import_modify_hybrid_data():
    popup = ttk.Toplevel()
    popup.title("Hybrid Data from Database")
    popup.geometry("1200x600")

    # { serial: {col_name: value, ...} }
    edit_tracker = {}

    # Patterns
    elog_number_format = re.compile(r"^ELOG-\d{6}$")           # ELOG-XXXXXX
    delivery_note_valtronic_format = re.compile(r"^\d{8}$")    # 8 digits

    # --- Search ---
    search_frame = ttk.Frame(popup)
    search_frame.pack(fill="x", padx=10, pady=5)
    ttk.Label(search_frame, text="Search Hybrid:", font=("Segoe UI", 11)).pack(side="left", padx=5)
    search_entry = ttk.Entry(search_frame, width=40)
    search_entry.pack(side="left", padx=5)
    search_button = ttk.Button(search_frame, text="Search", bootstyle="primary")
    search_button.pack(side="left", padx=5)

    # --- Table ---
    tree_frame = ttk.Frame(popup); tree_frame.pack(fill="both", expand=True)
    tree = ttk.Treeview(tree_frame, show="headings", selectmode="browse")
    tree.pack(side="left", fill="both", expand=True)
    vsb = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=vsb.set); vsb.pack(side="right", fill="y")

    columns = []; all_rows = []

    def fetch_data(search_value=None):
        nonlocal columns, all_rows
        try:
            conn = data_base_connection(); cursor = conn.cursor()
            if search_value:
                cursor.execute("""
                    SELECT * FROM shipping_tool.shipping_data
                    WHERE serial_number ILIKE %s
                    ORDER BY serial_number
                    LIMIT 100
                """, (f"%{search_value}%",))
            else:
                cursor.execute("""
                    SELECT * FROM shipping_tool.shipping_data
                    ORDER BY serial_number
                    LIMIT 100
                """)
            all_rows = cursor.fetchall()
            columns = [d[0] for d in cursor.description]
            conn.close()
        except Exception as e:
            Messagebox.show_error(f"Database error:\n{str(e)}"); popup.destroy(); return

        tree.delete(*tree.get_children()); tree["columns"] = columns
        for col in columns:
            tree.heading(col, text=col); tree.column(col, width=140)
        for row in all_rows:
            tree.insert("", tk.END, values=row)

    # --- Date helpers (DD.MM.YYYY like first button) ---
    def ymd_to_dmy_str(val):
        if val in (None, ""): return ""
        s = str(val)
        m = re.match(r"^(\d{4})-(\d{2})-(\d{2})$", s)
        if m: return f"{m.group(3)}.{m.group(2)}.{m.group(1)}"
        return s

    def dmy_to_iso(val):
        return datetime.strptime(val, "%d.%m.%Y").date().isoformat()

    def mask_date_entry(entry):
        raw = re.sub(r"\D", "", entry.get())[:8]
        out = raw
        if len(raw) > 2: out = raw[:2] + "." + raw[2:]
        if len(raw) > 4: out = out[:5] + "." + out[5:]
        if entry.get() != out:
            pos = entry.index(tk.INSERT)
            entry.delete(0, tk.END); entry.insert(0, out)
            try:
                entry.icursor(min(len(out), pos + (1 if pos in (2, 5) else 0)))
            except Exception:
                entry.icursor(tk.END)

    def on_double_click(event):
        item_id = tree.identify_row(event.y)
        col_id = tree.identify_column(event.x)
        if not item_id or not col_id: return
        col_index = int(col_id.replace("#", "")) - 1
        col_name = columns[col_index]  # actual DB column name

        editable_bool_cols = {"allowed_to_ship", "released_for_shipment"}
        editable_date_cols = {"reception_date"}
        editable_text_cols = {
            "delivery_number",            # PS### / 2S###
            "elog_number",                # ELOG-XXXXXX
            "delivery_note_valtronic",    # 8 digits
            "valtronic_lot_code",         # 10 digits
        }

        bbox = tree.bbox(item_id, col_id)
        if not bbox: return
        x, y, width, height = bbox

        current_values = list(tree.item(item_id, "values"))
        current_value = current_values[col_index]
        serial = current_values[columns.index("serial_number")]

        # --- Booleans ---
        if col_name in editable_bool_cols:
            def to_bool_str(v):
                if isinstance(v, bool): return "True" if v else "False"
                return "True" if str(v).strip().lower() in ("true","t","1","yes","y") else "False"
            combo = ttk.Combobox(popup, values=["True","False"], width=10, state="readonly")
            combo.set(to_bool_str(current_value))
            combo.place(x=x + tree.winfo_rootx() - popup.winfo_rootx(),
                        y=y + tree.winfo_rooty() - popup.winfo_rooty(),
                        width=width, height=height)
            def commit_bool(_):
                new_value = combo.get()
                current_values[col_index] = new_value; tree.item(item_id, values=current_values)
                edit_tracker.setdefault(serial, {})[col_name] = (new_value == "True")
                combo.destroy()
            combo.bind("<<ComboboxSelected>>", commit_bool); combo.focus_set(); return

        # --- Text fields with validation ---
        def make_entry(initial=""):
            entry = ttk.Entry(popup)
            entry.insert(0, "" if initial is None else str(initial))
            entry.place(x=x + tree.winfo_rootx() - popup.winfo_rootx(),
                        y=y + tree.winfo_rooty() - popup.winfo_rooty(),
                        width=width, height=height)
            entry.focus_set(); entry.icursor(tk.END)
            return entry

        if col_name == "delivery_number":
            entry = make_entry(current_value)
            def commit():
                new = entry.get().strip().upper()
                if not delivery_number_format.match(new):
                    Messagebox.show_error("Delivery number must be 'PS###' or '2S###' (e.g., PS012, 2S345)."); entry.focus_set(); return
                current_values[col_index] = new; tree.item(item_id, values=current_values)
                edit_tracker.setdefault(serial, {})[col_name] = new; entry.destroy()
            entry.bind("<Return>",   lambda e: (commit(), "break"))
            entry.bind("<Escape>",   lambda e: entry.destroy())
            entry.bind("<FocusOut>", lambda e: commit())
            return

        if col_name == "elog_number":
            entry = make_entry(current_value)
            def commit():
                new = entry.get().strip().upper()
                if new == "":
                    current_values[col_index] = ""; tree.item(item_id, values=current_values)
                    edit_tracker.setdefault(serial, {})[col_name] = None; entry.destroy(); return
                if not elog_number_format.match(new):
                    Messagebox.show_error("E-log must be ELOG-XXXXXX (exactly 6 digits)."); entry.focus_set(); return
                current_values[col_index] = new; tree.item(item_id, values=current_values)
                edit_tracker.setdefault(serial, {})[col_name] = new; entry.destroy()
            entry.bind("<Return>",   lambda e: (commit(), "break"))
            entry.bind("<Escape>",   lambda e: entry.destroy())
            entry.bind("<FocusOut>", lambda e: commit())
            return

        if col_name == "delivery_note_valtronic":
            entry = make_entry(current_value)
            def commit():
                new = entry.get().strip()
                if new == "":
                    current_values[col_index] = ""; tree.item(item_id, values=current_values)
                    edit_tracker.setdefault(serial, {})[col_name] = None; entry.destroy(); return
                if not delivery_note_valtronic_format.match(new):
                    Messagebox.show_error("Delivery note number (Valtronic) must be 8 digits."); entry.focus_set(); return
                current_values[col_index] = new; tree.item(item_id, values=current_values)
                edit_tracker.setdefault(serial, {})[col_name] = new; entry.destroy()
            entry.bind("<Return>",   lambda e: (commit(), "break"))
            entry.bind("<Escape>",   lambda e: entry.destroy())
            entry.bind("<FocusOut>", lambda e: commit())
            return

        if col_name == "valtronic_lot_code":
            entry = make_entry(current_value)
            def commit():
                new = entry.get().strip()
                if new == "":
                    current_values[col_index] = ""; tree.item(item_id, values=current_values)
                    edit_tracker.setdefault(serial, {})[col_name] = None; entry.destroy(); return
                if not valtronic_lot_code_format.match(new):
                    Messagebox.show_error("Valtronic lot code must be 10 digits."); entry.focus_set(); return
                current_values[col_index] = new; tree.item(item_id, values=current_values)
                edit_tracker.setdefault(serial, {})[col_name] = new; entry.destroy()
            entry.bind("<Return>",   lambda e: (commit(), "break"))
            entry.bind("<Escape>",   lambda e: entry.destroy())
            entry.bind("<FocusOut>", lambda e: commit())
            return

        # --- reception_date (DD.MM.YYYY masked; saved as ISO) ---
        if col_name in {"reception_date"}:
            entry = make_entry(ymd_to_dmy_str(current_value))
            entry.bind("<KeyRelease>", lambda e: mask_date_entry(entry))
            def commit():
                new_disp = entry.get().strip()
                if new_disp == "":
                    Messagebox.show_error("Date cannot be empty. Use DD.MM.YYYY."); entry.focus_set(); return
                try:
                    iso = dmy_to_iso(new_disp)
                except ValueError:
                    Messagebox.show_error("Invalid date. Use DD.MM.YYYY (e.g., 05.09.2025)."); entry.focus_set(); return
                current_values[col_index] = new_disp; tree.item(item_id, values=current_values)
                edit_tracker.setdefault(serial, {})[col_name] = iso; entry.destroy()
            entry.bind("<Return>",   lambda e: (commit(), "break"))
            entry.bind("<Escape>",   lambda e: entry.destroy())
            entry.bind("<FocusOut>", lambda e: commit())
            return

    def save_changes():
        # Safety: ensure any in-place editor commits before proceeding
        try:
            popup.focus_force()
            popup.update_idletasks()
        except Exception:
            pass

        if not edit_tracker:
            Messagebox.show_info("No changes to save."); return
        try:
            conn = data_base_connection(); cursor = conn.cursor()
            for serial, changes in edit_tracker.items():
                sets, params = [], []
                for key, value in changes.items():
                    sets.append(f"{key} = %s"); params.append(value)
                if not sets: continue
                params.append(serial)
                sql = f"""UPDATE shipping_tool.shipping_data
                          SET {", ".join(sets)}
                          WHERE serial_number = %s"""
                cursor.execute(sql, params)
            conn.commit(); conn.close()
            Messagebox.show_info("Changes saved successfully.")
            edit_tracker.clear(); fetch_data(search_entry.get().strip())
        except Exception as e:
            Messagebox.show_error(f"Failed to save changes:\n{str(e)}")

    search_button.config(command=lambda: fetch_data(search_entry.get().strip()))
    tree.bind("<Double-1>", on_double_click)
    fetch_data()

    # --- Helper: one dialog showing a list + multiple buttons; returns a key like "continue"/"skip"/"replace"/None ---
    def show_list_dialog(title, header, items, buttons):
        win = ttk.Toplevel(popup)
        win.title(title); win.geometry("600x420")
        win.transient(popup); win.grab_set()

        ttk.Label(win, text=header, wraplength=560, justify="left").pack(padx=10, pady=10, anchor="w")

        frame = ttk.Frame(win); frame.pack(fill="both", expand=True, padx=10)
        lb = tk.Listbox(frame, height=12)
        lb.pack(side="left", fill="both", expand=True)
        sb = ttk.Scrollbar(frame, orient="vertical", command=lb.yview); sb.pack(side="right", fill="y")
        lb.configure(yscrollcommand=sb.set)

        for it in items:
            lb.insert(tk.END, str(it))

        btn_row = ttk.Frame(win); btn_row.pack(pady=10)
        result = {"value": None}
        for text, key, style in buttons:
            ttk.Button(btn_row, text=text, bootstyle=style,
                       command=lambda k=key: (result.update(value=k), win.destroy())).pack(side="left", padx=8)

        win.wait_window()
        return result["value"]

    # CSV import with warnings
    def import_csv_into_database():
        file_path = filedialog.askopenfilename(title="Select CSV File", filetypes=[("CSV Files", "*.csv")])
        if not file_path: return

        try:
            df = pd.read_csv(file_path)
        except Exception as e:
            Messagebox.show_error(f"Could not read CSV:\n{str(e)}"); return

        if 'serial_number' not in df.columns or 'allowed_to_ship' not in df.columns:
            Messagebox.show_error("CSV must contain 'serial_number' and 'allowed_to_ship' columns."); return

        # Validate serial format
        df['serial_number'] = df['serial_number'].astype(str).str.strip()
        invalid_serials = df[~df['serial_number'].str.match(serial_number_format)]
        if not invalid_serials.empty:
            items = invalid_serials['serial_number'].tolist()
            choice = show_list_dialog(
                "Invalid Serials Found",
                f"{len(items)} invalid serial(s) found. These rows will be skipped unless you cancel.\n",
                items,
                buttons=[("Skip invalids & continue", "continue", "warning"),
                         ("Cancel import", "cancel", "secondary")]
            )
            if choice != "continue":
                return
            df = df[df['serial_number'].str.match(serial_number_format)]

        if df.empty:
            Messagebox.show_info("No valid data to import."); return

        try:
            conn = data_base_connection(); cursor = conn.cursor()
            cursor.execute("SELECT serial_number FROM shipping_tool.shipping_data")
            existing_serials = set(row[0] for row in cursor.fetchall())

            df['allowed_to_ship'] = df['allowed_to_ship'].astype(str).str.strip().str.lower().isin(
                ["true","t","1","yes","y"])

            duplicates_df = df[df['serial_number'].isin(existing_serials)]
            new_df        = df[~df['serial_number'].isin(existing_serials)]

            to_insert = [(r['serial_number'], r['allowed_to_ship']) for _, r in new_df.iterrows()]
            to_update = []

            # One dialog for all duplicates
            skipped_dups = 0
            if not duplicates_df.empty:
                items = duplicates_df['serial_number'].tolist()
                choice = show_list_dialog(
                    "Duplicates Found",
                    f"{len(items)} serial(s) already exist in the database.\n"
                    f"Choose what to do with ALL duplicates:",
                    items,
                    buttons=[("Update all duplicates", "replace", "success"),
                             ("Skip all duplicates", "skip", "secondary"),
                             ("Cancel import", "cancel", "danger")]
                )
                if choice == "replace":
                    to_update = [(r['allowed_to_ship'], r['serial_number']) for _, r in duplicates_df.iterrows()]
                elif choice == "skip":
                    skipped_dups = len(items)
                else:
                    return

            # Execute database ops
            if to_insert:
                cursor.executemany("""
                    INSERT INTO shipping_tool.shipping_data (serial_number, allowed_to_ship)
                    VALUES (%s, %s)
                """, to_insert)
            if to_update:
                cursor.executemany("""
                    UPDATE shipping_tool.shipping_data
                    SET allowed_to_ship = %s
                    WHERE serial_number = %s
                """, to_update)

            conn.commit(); conn.close()

            Messagebox.show_info(
                "Import complete.\n"
                f"Inserted: {len(to_insert)}\n"
                f"Updated: {len(to_update)}\n"
                f"Skipped invalid: {len(invalid_serials)}\n"
                f"Skipped duplicates: {skipped_dups}"
            )
            fetch_data(search_entry.get().strip())  # refresh

        except Exception as e:
            Messagebox.show_error(f"Database operation failed:\n{str(e)}")

    # Buttons
    btn_frame = ttk.Frame(popup); btn_frame.pack(pady=10)
    ttk.Button(btn_frame, text="Import from CSV", command=import_csv_into_database, bootstyle="primary").pack(side="left", padx=10)
    ttk.Button(btn_frame, text="Save Changes", command=save_changes, bootstyle="success").pack(side="left", padx=10)

    def delete_selected():
        selected = tree.selection()
        if not selected: Messagebox.show_info("No rows selected."); return
        serials = [tree.item(i)["values"][columns.index("serial_number")] for i in selected]
        if not Messagebox.okcancel(f"Delete {len(serials)} serial(s)? This cannot be undone.", title="Confirm Deletion"):
            return
        try:
            conn = data_base_connection(); cursor = conn.cursor()
            cursor.executemany("""DELETE FROM shipping_tool.shipping_data WHERE serial_number = %s""", [(s,) for s in serials])
            conn.commit(); conn.close()
            for i in selected: tree.delete(i)
            Messagebox.show_info("Selected serial(s) deleted successfully.")
        except Exception as e:
            Messagebox.show_error(f"Failed to delete:\n{str(e)}")




#-----------------------
#THIRD BUTTON
#-----------------------

#_______________________________SET READY FOR SHIPMENT___________________________________
def set_ready_for_shipment():
    popup = ttk.Toplevel()
    popup.title("Set Hybrids Ready for Shipment")
    popup.geometry("1000x600")

    tree = ttk.Treeview(popup, show="headings", selectmode="extended")
    tree.pack(side="left", fill="both", expand=True)

    vsb = ttk.Scrollbar(popup, orient="vertical", command=tree.yview)
    tree.configure(yscrollcommand=vsb.set)
    vsb.pack(side="right", fill="y")

    def fetch_data():
        try:
            conn = data_base_connection()
            cursor = conn.cursor()

            cursor.execute("""
                SELECT serial_number, allowed_to_ship, released_for_shipment, delivery_number, reception_date
                FROM shipping_tool.shipping_data
                WHERE allowed_to_ship = TRUE
            """)
            rows = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]

            tree["columns"] = columns
            tree.delete(*tree.get_children())

            for col in columns:
                tree.heading(col, text=col)
                tree.column(col, width=160)

            for row in rows:
                tree.insert("", tk.END, values=row)

            conn.close()
        except Exception as e:
            Messagebox.show_error(f"Database fetch error:\n{str(e)}")
            popup.destroy()

    def mark_as_ready():
        selected_items = tree.selection()
        if not selected_items:
            Messagebox.show_info("No rows selected.")
            return

        serials = [
            tree.item(item)["values"][0]  # serial_number is first column
            for item in selected_items
        ]

        try:
            conn = data_base_connection()
            cursor = conn.cursor()

            cursor.executemany("""
                UPDATE shipping_tool.shipping_data
                SET released_for_shipment = TRUE
                WHERE serial_number = %s
            """, [(s,) for s in serials])

            conn.commit()
            conn.close()
            Messagebox.show_info("Selected hybrids marked as ready for shipment.")
            fetch_data()  # Refresh
        except Exception as e:
            Messagebox.show_error(f"Failed to update:\n{str(e)}")

    btn_frame = ttk.Frame(popup)
    btn_frame.pack(pady=10)

    #ttk.Button(btn_frame, text="Mark as Ready for Shipment", command=mark_as_ready, bootstyle="success").pack(side="left", padx=10)
    ttk.Button(btn_frame, text="Close", command=popup.destroy, bootstyle="secondary").pack(side="right", padx=10)

    fetch_data()



#________Run_______________________________________________________________________
def launch_front_page():
    root = ttk.Window(themename="darkly")
    root.title("Hybrid Thingy")
    root.geometry("1000x500")

    button_frame = ttk.Frame(root)
    button_frame.pack(expand=True)

    b1 = ttk.Button(button_frame, text="Reception in\nB02", width=20, bootstyle="success", command=reception_b02_popup)
    b2 = ttk.Button(button_frame, text="Import or Modify Hybrid\nData", width=25, bootstyle="info", command=import_modify_hybrid_data)
    b3 = ttk.Button(button_frame, text="Set ready for\nShipment", width=20, bootstyle="warning", command=set_ready_for_shipment)



    b1.grid(row=0, column=0, padx=20, pady=20)
    b2.grid(row=0, column=1, padx=20, pady=20)
    b3.grid(row=0, column=2, padx=20, pady=20)

    root.mainloop()

launch_front_page()